# iRecycle Digital Database Creation

## Verify that we are in the correct database "irecycledigitaldatabase"

In [1]:
SELECT DB_NAME() AS CurrentDatabase;

(1 row affected)

Total execution time: 00:00:00.028

CurrentDatabase
irecycledigitaldatabase


## 0\. Create recycling database schema

In [2]:
-- Create schema for better organization
CREATE SCHEMA recycling;
GO 

: Msg 2714, Level 16, State 6, Line 2
There is already an object named 'recycling' in the database.

: Msg 2759, Level 16, State 0, Line 2
CREATE SCHEMA failed due to previous errors.

Total execution time: 00:00:00.044

## 1\. Create recycling business table

In [3]:
CREATE TABLE recycling.Businesses (
    BusinessID INT IDENTITY(1,1) PRIMARY KEY,
    Name NVARCHAR(255) NOT NULL,
    FormattedAddress NVARCHAR(500) NOT NULL,
    Latitude DECIMAL(10, 8),
    Longitude DECIMAL(11, 8),
    PhoneNumber NVARCHAR(50),
    Website NVARCHAR(500),
    Rating DECIMAL(3, 2),
    PlaceID NVARCHAR(255),  -- Google Places ID
    DateAdded DATETIME2 DEFAULT SYSUTCDATETIME(),
    LastUpdated DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL,
    SearchVector NVARCHAR(MAX), -- For full-text search
    ServiceKeywords NVARCHAR(MAX) -- JSON array of keywords
);

-- Create indexes for better search performance
CREATE INDEX IX_Businesses_Location ON recycling.Businesses(Latitude, Longitude);
CREATE INDEX IX_Businesses_Name ON recycling.Businesses(Name); 


CREATE TABLE recycling.AddressComponents (
    AddressComponentID INT PRIMARY KEY IDENTITY(1,1),
    BusinessID INT,
    StreetAddress NVARCHAR(255),
    City NVARCHAR(100),
    State NVARCHAR(100),
    PostalCode NVARCHAR(20),
    Country NVARCHAR(100),
    FOREIGN KEY (BusinessID) REFERENCES recycling.Businesses(BusinessID)
);

Commands completed successfully.

Total execution time: 00:00:00.043

## 2\. Create recycling business services table

In [4]:
CREATE TABLE recycling.BusinessServices (
    ServiceID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    ServiceName NVARCHAR(100) NOT NULL,
    Description NVARCHAR(500),
    IsBookingEnabled BIT DEFAULT 0,
    PriceInfo NVARCHAR(MAX), -- JSON field for flexible pricing structure
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME()
); 

-- Create an index for the BusinessServices table
CREATE INDEX IX_BusinessServices_BusinessID ON recycling.BusinessServices(BusinessID);

Commands completed successfully.

Total execution time: 00:00:00.039

## 3\. Create recycling business hours table

In [5]:
CREATE TABLE recycling.BusinessHours (
    HoursID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    DayOfWeek TINYINT,  -- 0 = Sunday, 1 = Monday, etc.
    OpenTime TIME,
    CloseTime TIME,
    IsClosed BIT DEFAULT 0
);

-- Create an index for the BusinessHours table
CREATE INDEX IX_BusinessHours_BusinessID ON recycling.BusinessHours(BusinessID);

Commands completed successfully.

Total execution time: 00:00:00.052

## 4\. Create recycling materials table

In [6]:
CREATE TABLE recycling.Materials (
    MaterialID INT IDENTITY(1,1) PRIMARY KEY,
    CategoryName NVARCHAR(50), -- e.g., 'plastic', 'metal', etc.
    Description NVARCHAR(500), -- Description of the material
    CO2Savings DECIMAL(10, 2)  -- CO2 savings per kg
    CONSTRAINT UQ_Description UNIQUE (Description)
); 

Commands completed successfully.

Total execution time: 00:00:00.037

## <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">5. Create recycling business materials table</span>

In [7]:
CREATE TABLE recycling.BusinessMaterials (
    BusinessID INT,
    MaterialID INT,
    CategoryName NVARCHAR(50),
    Description NVARCHAR(500),
    IsVerified BIT DEFAULT 0,
    VerificationSource NVARCHAR(50),
    DateVerified DATETIME2,
    PRIMARY KEY (BusinessID, MaterialID),
    FOREIGN KEY (BusinessID) REFERENCES recycling.Businesses(BusinessID),
    FOREIGN KEY (MaterialID) REFERENCES recycling.Materials(MaterialID)
);

-- Create additional index for the BusinessMaterials table
CREATE INDEX IX_BusinessMaterials_BusinessID ON recycling.BusinessMaterials(BusinessID);
CREATE INDEX IX_BusinessMaterials_CategoryName ON recycling.Materials(CategoryName);

Commands completed successfully.

Total execution time: 00:00:00.058

## 6\. Create business account table

In [8]:
CREATE TABLE recycling.BusinessAccounts (
    AccountID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT UNIQUE FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    Email NVARCHAR(255) NOT NULL UNIQUE,
    IsEmailVerified BIT DEFAULT 0,
    VerificationDate DATETIME2,
    LastLoginDate DATETIME2,
    AccountStatus NVARCHAR(20) DEFAULT 'PENDING', -- PENDING, ACTIVE, SUSPENDED
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME()
); 

-- Create an index for the BusinessAccounts table
CREATE INDEX IX_BusinessAccounts_BusinessID ON recycling.BusinessAccounts(BusinessID);

Commands completed successfully.

Total execution time: 00:00:00.040

## 7. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Create recycling user table</span>

In [9]:
CREATE TABLE recycling.Users (
    UserID INT IDENTITY(1,1) PRIMARY KEY,
    Email NVARCHAR(255) NOT NULL UNIQUE,
    PhoneNumber NVARCHAR(50),
    FirstName NVARCHAR(100),
    LastName NVARCHAR(100),
    DefaultAddress NVARCHAR(500),
    DefaultLatitude DECIMAL(10, 8),
    DefaultLongitude DECIMAL(11, 8),
    UserRole NVARCHAR(50), -- Residential, Commercial, Waste Picker
    IsPhoneVerified BIT DEFAULT 0,
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
); 

-- Create an index for the Users table
CREATE INDEX IX_Users_Email ON recycling.Users(Email);

Commands completed successfully.

Total execution time: 00:00:00.040

## 8. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Create user subscription table</span>

In [10]:
CREATE TABLE recycling.UserSubscriptions (
    SubscriptionID INT IDENTITY(1,1) PRIMARY KEY,
    UserID INT FOREIGN KEY REFERENCES recycling.Users(UserID),
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    NotificationsEnabled BIT DEFAULT 1,
    SubscriptionDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL,
    UNIQUE (UserID, BusinessID)
); 

-- Create an index for the UserSubscriptions table
CREATE INDEX IX_UserSubscriptions_UserID ON recycling.UserSubscriptions(UserID);
CREATE INDEX IX_UserSubscriptions_BusinessID ON recycling.UserSubscriptions(BusinessID);

Commands completed successfully.

Total execution time: 00:00:00.044

## 9. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Insert recycling materials</span>

In [11]:
INSERT INTO recycling.Materials (CategoryName, Description, CO2Savings) VALUES
('paper', 'Corrugated Containers', 5.58),
('paper', 'Magazines/third-class mail', 8.57),
('paper', 'Newspaper', 4.68),
('paper', 'Office Paper', 7.95),
('paper', 'Phonebooks', 6.17),
('paper', 'Textbooks', 9.02),
('paper', 'Mixed Paper (general)', 6.07),
('paper', 'Mixed Paper (primarily)', 6.00),
('paper', 'Mixed Paper (primarily from Food Waste)', 3.66),
('organic', 'Food Waste (non-meat)', 0.76),
('organic', 'Food Waste (meat only)', 15.1),
('organic', 'Beef', 30.9),
('organic', 'Poultry', 2.45),
('organic', 'Grains', 0.62),
('organic', 'Bread', 0.66),
('organic', 'Fruits and Vegetables', 0.44),
('organic', 'Dairy Products', 1.75),
('plastic', 'HDPE', 1.42),
('plastic', 'LDPE', 1.80),
('plastic', 'PET', 2.17),
('plastic', 'LLDPE', 1.58),
('plastic', 'PP', 1.00), -- Adjust as necessary
('plastic', 'PS', 2.50),
('plastic', 'PVC', 1.93),
('plastic', 'Mixed Plastics', 1.87),
('plastic', 'PLA', 2.45),
('electronics', 'Desktop CPUs', 20.80),
('electronics', 'Portable Electronic Devices', 29.83),
('electronics', 'Flat-Panel Displays', 24.19),
('electronics', 'Electronic Peripherals', 10.32),
('electronics', 'Hard-Copy Devices', 7.65),
('electronics', 'Mixed Electronics', 20.79),
('metal', 'Aluminum Cans', 4.80),
('metal', 'Aluminum Ingot', 7.48),
('metal', 'Steel Cans', 3.03),
('metal', 'Copper Wire', 6.72),
('metal', 'Mixed Metals', 3.65),
('glass', 'Glass', 0.53),
('construction', 'Asphalt Concrete', 0.19),
('construction', 'Asphalt Shingles', 0.19),
('construction', 'Carpet', 3.68),
('construction', 'Clay Bricks', 0.27),
('construction', 'Dimensional Lumber', 2.11),
('construction', 'Drywall', 0.00), -- Adjust as necessary
('construction', 'Fiberglass Insulation', 0.38),
('construction', 'Medium-density Fiberboard', 3.05),
('construction', 'Structural Steel', 1.67),
('construction', 'Vinyl Flooring', 0.58),
('construction', 'Wood Flooring', 4.11),
('tires', 'Tires', 4.30);

(50 rows affected)

Total execution time: 00:00:00.042

# DONT CREATE THESE TABLES YET!

In [ ]:
CREATE TABLE recycling.CommunicationPreferences (
    UserID INT PRIMARY KEY FOREIGN KEY REFERENCES recycling.Users(UserID),
    EmailNotifications BIT DEFAULT 1,
    PushNotifications BIT DEFAULT 0,
    SMSNotifications BIT DEFAULT 0,
    NotificationFrequency NVARCHAR(20) DEFAULT 'DAILY', -- Weekly, Monthly
    PreferredChannels NVARCHAR(MAX) -- JSON array of preferred communication methods
); 

In [ ]:
CREATE TABLE recycling.RecyclingAlerts (
    AlertID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    AlertType NVARCHAR(50), -- 'SCHEDULE_CHANGE', 'NEW_SERVICE', 'CONTAMINATION_FEEDBACK'
    Title NVARCHAR(200),
    Content NVARCHAR(MAX),
    Severity NVARCHAR(20),
    ValidFrom DATETIME2,
    ValidTo DATETIME2,
    CreatedAt DATETIME2 DEFAULT SYSUTCDATETIME()
);

In [ ]:
CREATE TABLE recycling.CollectionBookings (
    BookingID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    UserID INT FOREIGN KEY REFERENCES recycling.Users(UserID),
    BookingType NVARCHAR(20), -- 'PICKUP', 'DROPOFF'
    ScheduledDate DATE,
    ScheduledTimeSlot NVARCHAR(50),
    Status NVARCHAR(20), -- 'PENDING', 'CONFIRMED', 'COMPLETED', 'CANCELLED'
    Address NVARCHAR(500),
    Latitude DECIMAL(10, 8),
    Longitude DECIMAL(11, 8),
    Notes NVARCHAR(500),
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    ModifiedDate DATETIME2 DEFAULT SYSUTCDATETIME(),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
);

-- Create an index for the CollectionBookings table
CREATE INDEX IX_CollectionBookings_BusinessID ON recycling.CollectionBookings(BusinessID);
CREATE INDEX IX_CollectionBookings_UserID ON recycling.CollectionBookings(UserID);

In [ ]:
CREATE TABLE recycling.WasteRecords (
    RecordID INT IDENTITY(1,1) PRIMARY KEY,
    BusinessID INT FOREIGN KEY REFERENCES recycling.Businesses(BusinessID),
    MaterialCategoryID INT FOREIGN KEY REFERENCES recycling.MaterialCategories(CategoryID),
    Quantity DECIMAL(10,2),
    Unit NVARCHAR(20),
    RecordDate DATE,
    Notes NVARCHAR(500),
    CreatedDate DATETIME2 DEFAULT SYSUTCDATETIME()
);

-- Create an index for the WasteRecords table
CREATE INDEX IX_WasteRecords_BusinessID ON recycling.WasteRecords(BusinessID);
CREATE INDEX IX_WasteRecords_MaterialCategoryID ON recycling.WasteRecords(MaterialCategoryID);

In [ ]:
CREATE TABLE recycling.MaterialTransaction (
    TransactionID INT IDENTITY(1,1) PRIMARY KEY,
    RecordID INT FOREIGN KEY REFERENCES recycling.WasteRecords(RecordID),
    CategoryID INT FOREIGN KEY REFERENCES recycling.MaterialCategories(CategoryID),
    Quantity DECIMAL(10,2),
    Unit NVARCHAR(20),
    PricePerUnit DECIMAL(10,2),
    TotalPrice DECIMAL(10,2),
    Notes NVARCHAR(500),
    IsActive BIT DEFAULT 1,
    DeletedAt DATETIME2 NULL,
    DeletedBy NVARCHAR(128) NULL
); 

-- Create an index for the MaterialTransaction table
CREATE INDEX IX_MaterialTransaction_RecordID ON recycling.MaterialTransaction(RecordID);
CREATE INDEX IX_MaterialTransaction_CategoryID ON recycling.MaterialTransaction(CategoryID);